In [2]:
import pandapower as pp
import numpy as np

In [3]:
microgrid = pp.create_empty_network(name="cub_campus")
high_voltage = 23
low_voltage = 13.8

# Xcel energy operates at 13.8kV for distribution
grid_bus1 = pp.create_bus(microgrid, name="GRID_bus1", vn_kv=high_voltage,geodata=(0,0))
#grid_bus2 = pp.create_bus(microgrid, name="GRID_bus2", vn_kv=high_voltage)
lv_bus1 = pp.create_bus(microgrid, name="LV_bus1", vn_kv=low_voltage, type="n",geodata=(0.5,0))
sb_bus1 = pp.create_bus(microgrid, name="SB_bus1", vn_kv=low_voltage, type="n",geodata=(1,0))
multidomain_bus = pp.create_bus(microgrid, name="MultiDomain_bus", vn_kv=0.4, type="n",geodata=(1,0.5))
multidomain_bus1 = pp.create_bus(microgrid, name="MultiDomain_bus1", vn_kv=low_voltage, type="n", geodata=(1,0.5))
central_bus = pp.create_bus(microgrid, name="CENTRAL_bus", vn_kv=low_voltage,geodata=(1.5,0))
pv_bus = pp.create_bus(microgrid, name="PV_bus", vn_kv=low_voltage,geodata=(2,0.5))
bess_bus = pp.create_bus(microgrid, name="BESS_bus", vn_kv=low_voltage, type="n",geodata=(2,1))
gen_bus = pp.create_bus(microgrid, name="Gen_bus", vn_kv=low_voltage, type="n",geodata=(0.5,0.5))
sec1 = pp.create_bus(microgrid, name="sec1", vn_kv=low_voltage, type="n",geodata=(3,1))
sec2 = pp.create_bus(microgrid, name="sec2", vn_kv=low_voltage, type="n",geodata=(3,-1))
load1 = pp.create_bus(microgrid, name="macky", vn_kv=low_voltage, type="n",geodata=(3.5,1.3))
load2 = pp.create_bus(microgrid, name="hellums", vn_kv=low_voltage, type="n",geodata=(3.5,0.7))
load3 = pp.create_bus(microgrid, name="stadium", vn_kv=low_voltage, type="n",geodata=(3,0))
load4 = pp.create_bus(microgrid, name="bookstore", vn_kv=low_voltage, type="n",geodata=(4,1))
load5 = pp.create_bus(microgrid, name="quad", vn_kv=low_voltage, type="n",geodata=(3.5,-0.7))
load6 = pp.create_bus(microgrid, name="ec", vn_kv=low_voltage, type="n",geodata=(3.5,-1.3))
load7 = pp.create_bus(microgrid, name="chw_plant", vn_kv=low_voltage, type="n",geodata=(4,-0.7))
load8 = pp.create_bus(microgrid, name="kitt", vn_kv=low_voltage, type="n",geodata=(4,-1.3))

buses = [grid_bus1, lv_bus1, sb_bus1, multidomain_bus1, central_bus, pv_bus, bess_bus, sec1, sec2, load1, load2, load3, load4, load5, load6, load7, load8, gen_bus]
conns = [[1,2],[2,4],[3,4],[4,5],[4,6],[4,7],[4,11],[4,8],[7,9],[7,12],[9,10],[10,12],[8,13],[8,15],[13,14],[14,16],[16,15],[3,17]]
for conn in conns:
    pp.create_line(microgrid, buses[conn[0]], buses[conn[1]], 0.5, std_type="N2XS(FL)2Y 1x300 RM/35 64/110 kV")

xcel_conn = pp.create_ext_grid(microgrid, grid_bus1, vm_pu=1.02, va_degree=50)
xcel_transformer = pp.create_transformer_from_parameters(microgrid, grid_bus1, lv_bus1, 2000, high_voltage, low_voltage, 0.25, 10, 0.48, 0.06)
chp_gen = pp.create_gen(microgrid, gen_bus, p_mw=3.5)
chp_transformer = pp.create_transformer_from_parameters(microgrid, multidomain_bus, multidomain_bus1, 2000, 0.4, low_voltage, 0.25, 10, 0.48, 0.06)
pv_gen = pp.create_gen(microgrid, pv_bus, p_mw=0)

ps = [347.648998,189.584781,4337.210930,1923.991871,3396.980054,796.892406,972.234648,1028.719053]
ls = [load1, load2, load3, load4, load5, load6, load7, load8]
for l in range(len(ls)):
    p = ps[l] * 0.001
    s = p * np.tan(np.arccos(np.random.uniform(0.85,0.95)))
    pp.create_load(microgrid, ls[l], p_mw=p, q_mvar=s, const_i_percent=1, const_z_percent=3)
    
pp.runpp(microgrid)

In [11]:
microgrid.res_ext_grid

,p_mw,q_mvar
0,9.660509,19.614711


In [4]:
v_b = 13800
s_b = 10**6 
z_b = (v_b**2)/s_b
microgrid.line['pu_r'] = microgrid.line['r_ohm_per_km'] / z_b
microgrid.line['pu_z'] = microgrid.line['x_ohm_per_km'] / z_b

In [5]:
import pandas as pd
x = pd.merge(microgrid.bus, microgrid.res_bus, left_index=True, right_index=True)

In [6]:
for i in ['vm_pu','va_degree']:
    x[f'data_record_{i[:2]}'] = f"parameter Real {i[:2]}_"+x['name']+"="+x[i].astype(str)+";"

In [7]:
print(x.to_string(columns=['data_record_vm'],index=False,header=False))
print(x.to_string(columns=['data_record_va'],index=False,header=False))

                     parameter Real vm_GRID_bus1=1.02;
         parameter Real vm_LV_bus1=1.0190576316131048;
         parameter Real vm_SB_bus1=1.0107576391520277;
 parameter Real vm_MultiDomain_bus=1.0009698495847619;
parameter Real vm_MultiDomain_bus1=1.0009998695928526;
     parameter Real vm_CENTRAL_bus=1.0024565618609051;
          parameter Real vm_PV_bus=0.9999999999999999;
        parameter Real vm_BESS_bus=1.0024573781627082;
         parameter Real vm_Gen_bus=0.9999999999999999;
            parameter Real vm_sec1=1.0015857307746139;
            parameter Real vm_sec2=1.0002596370701418;
           parameter Real vm_macky=1.0013043385168834;
         parameter Real vm_hellums=1.0011203791558325;
         parameter Real vm_stadium=1.0011359621986726;
       parameter Real vm_bookstore=1.0009942536074952;
            parameter Real vm_quad=0.9988582420760337;
              parameter Real vm_ec=0.9987657707873655;
       parameter Real vm_chw_plant=0.9994642199952037;
          

In [7]:
x = pd.merge(microgrid.load.drop(['p_mw','q_mvar'],axis=1), microgrid.res_load, left_index=True, right_index=True)
x['name'] = (x.index+1).astype(str)

In [8]:
for i in ['p_mw','q_mvar']:
    x[f'data_record_{i[:1]}'] = f"parameter Real {i[:1]}_load"+x['name']+"="+x[i].astype(str)+";"

In [9]:
print(x.to_string(columns=['data_record_p'],index=False,header=False))
print(x.to_string(columns=['data_record_q'],index=False,header=False))

 parameter Real p_load1=0.3476884613879555;
parameter Real p_load2=0.18960436975031572;
  parameter Real p_load3=4.337643015903549;
  parameter Real p_load4=1.924179200838282;
 parameter Real p_load5=3.3967634665305946;
 parameter Real p_load6=0.7968348557639645;
 parameter Real p_load7=0.9722112878049635;
  parameter Real p_load8=1.028657900034803;
  parameter Real q_load1=0.193751880346159;
parameter Real q_load2=0.08515620541406078;
  parameter Real q_load3=1.480572466770676;
 parameter Real q_load4=0.7571525872248411;
 parameter Real q_load5=1.9169154673838822;
parameter Real q_load6=0.47434610535510885;
 parameter Real q_load7=0.4094702965042356;
parameter Real q_load8=0.43070350818137665;


In [10]:
microgrid.line['name'] = microgrid.bus.iloc[microgrid.line['to_bus']]['name'].values + "_" + microgrid.bus.iloc[microgrid.line['from_bus']]['name'].values
microgrid.line['r'] = microgrid.line['length_km'] * microgrid.line['r_ohm_per_km']
microgrid.line['x'] = microgrid.line['length_km'] * microgrid.line['x_ohm_per_km']
microgrid.line['data_record_r'] = "parameter Real " + microgrid.line['name'] + "_r=" + microgrid.line['r_ohm_per_km'].astype(str) + ";"
microgrid.line['data_record_x'] = "parameter Real " + microgrid.line['name'] + "_x=" + microgrid.line['x_ohm_per_km'].astype(str) + ";"

In [11]:
print(microgrid.line.to_string(columns=['data_record_r'],index=False,header=False))
print(microgrid.line.to_string(columns=['data_record_x'],index=False,header=False))

            parameter Real SB_bus1_LV_bus1_r=0.06;
        parameter Real CENTRAL_bus_SB_bus1_r=0.06;
parameter Real CENTRAL_bus_MultiDomain_bus_r=0.06;
         parameter Real PV_bus_CENTRAL_bus_r=0.06;
       parameter Real BESS_bus_CENTRAL_bus_r=0.06;
           parameter Real sec1_CENTRAL_bus_r=0.06;
        parameter Real stadium_CENTRAL_bus_r=0.06;
           parameter Real sec2_CENTRAL_bus_r=0.06;
                 parameter Real macky_sec1_r=0.06;
             parameter Real bookstore_sec1_r=0.06;
              parameter Real hellums_macky_r=0.06;
          parameter Real bookstore_hellums_r=0.06;
                  parameter Real quad_sec2_r=0.06;
             parameter Real chw_plant_sec2_r=0.06;
                    parameter Real ec_quad_r=0.06;
                    parameter Real kitt_ec_r=0.06;
             parameter Real chw_plant_kitt_r=0.06;
    parameter Real Gen_bus_MultiDomain_bus_r=0.06;
            parameter Real SB_bus1_LV_bus1_x=0.144;
        parameter Real CENTRAL